<a href="https://colab.research.google.com/github/luca-arts/seeingtheimperceptible/blob/main/notebooks/colorGrading/tests/LPTN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Color grading via LPTN

[https://github.com/csjliang/LPTN](https://github.com/csjliang/LPTN)


In [39]:
# we'll link the dataset from next-cloud
import os
!apt install -y davfs2  > /dev/null

nextcloud = "/content/nextcloud"
Nextcloud_URL = "https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/" #@param {type:"string"}

os.makedirs(nextcloud, exist_ok=True)


if os.path.isfile("/etc/fstab"):
  os.remove("/etc/fstab")
with open("/etc/fstab" , "a") as f:
  f.write(Nextcloud_URL + " " + nextcloud + " davfs user,rw,auto 0 0")
!mount {nextcloud}

# open existing text file
txt = "/etc/fstab"
viewTxt = open(txt, "r")
txtContent = viewTxt.read()
print("txtContent: {}".format(txtContent))



Please enter the username to authenticate with server
https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ or hit enter for none.
  Username: colab
Please enter the password to authenticate user colab with server
https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ or hit enter for none.
  Password:  
/sbin/mount.davfs: https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ is already mounted on /content/nextcloud
txtContent: https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ /content/nextcloud davfs user,rw,auto 0 0


## 2. create I/O folders

In [40]:
import shutil
from google.colab import files

def create_io(database, topic, library):
  # clean and rebuild the image folders
  input_folder = database+'/'+topic
  # '/content/nextcloud/colorGrading'
  # if os.path.exists(input_folder):
  #   shutil.rmtree(input_folder)
  # os.makedirs(input_folder)

  output_folder = database+'/'+topic+'/'+library# '/content/nextcloud/colorGrading/output'
  if os.path.exists(output_folder):
    shutil.rmtree(output_folder)
  os.makedirs(output_folder)
  return input_folder, output_folder

input_folder, output_folder = create_io('/content/nextcloud','colorGrading','LPTN')

In [ ]:
os.listdir(input_folder)

In [ ]:
print(output_folder)

In [ ]:
%ls

## 3. Clone the Git repository

In [46]:
# clone the repository
root_path = '/content/LPTN'

if not os.path.exists(root_path):
  !git clone https://github.com/csjliang/LPTN

## 4. install dependencies

In [ ]:
!pip install -r requirement.txt

## 5. Pretrained model download

In [48]:
# check first if exist, otherwise create path
pretrained_pth = root_path + '/experiments/pretrained_models/net_g_FiveK_numhigh3.pth'

if not os.path.exists(pretrained_pth):
  !gdown --id 11yuFgHqZe9e4OheJ9YHp9lS-4kQC1Fml \
          -O pretrained_models/net_g_FiveK_numhigh3.pth

## 6. install pyYAML and update the YAML file with correct path for pretrained_models

geen idee of we dit nodig hebben, maar heb zelf deze code erin gezet om te checken of het path effectief naar de goeie folder wijst.


In [ ]:
!pip install pyYAML

import yaml
 
def read_yaml():
    """ A function to read YAML file"""
    with open( root_path + '/options/test/LPTN/test_FiveK.yml') as f:
        config = yaml.safe_load(f)
 
    return config

read_yaml()

## 7. dataset downloaden

download dataset

In [ ]:
import os
import zipfile

# define folders
output_path_dataset = root_path +'/datasets'
output_path_FiveK = output_path_dataset +'/FiveK'
input_path = nextcloud + '/colorGrading/datasets/FiveK_480p.zip'

# make folders
os.makedirs(output_path_dataset, exist_ok=True)
os.makedirs(output_path_FiveK, exist_ok=True)

#!unzip "/content/drive/path/input_file_name.zip" -d "/content/drive/path/output_folder/"
!unzip {input_path} -d {output_path_FiveK}

%ls

create lmdb

In [56]:
#PYTHONPATH="./:${PYTHONPATH}" python scripts/data_preparation/create_lmdb.py
!python3 {root_path} + '/scripts/data_preparation/create_lmdb.py'

/usr/bin/python3: can't find '__main__' module in '/content/LPTN'


## 8. Evaluation

yml file test

In [ ]:
# PYTHONPATH="./:${PYTHONPATH}" CUDA_VISIBLE_DEVICES=0 python codes/test.py -opt options/test/LPTN/test_FiveK.yml
!python3 /content/LPTN/codes/test.py -opt options/test/LPTN/test_FiveK.yml

speed testing

In [ ]:
# PYTHONPATH="./:${PYTHONPATH}" CUDA_VISIBLE_DEVICES=0 python codes/test_speed.py -opt options/test/LPTN/test_speed_FiveK.yml
!python3 /content/LPTN/codes/test_speed.py -opt options/test/LPTN/test_speed_FiveK.yml